In [4]:
import pandas as pd 
import numpy as np 
import json
import os 
import random

In [22]:
class Node: 
    
    def __init__(self, data): 
        self.data = data 
        self.uid = random.getrandbits(128)
        self.node = {
            'data': data, 
            'uid': self.uid
        } 
    
    def alter(self, key, val): 
        "alter a nodes data"  
        
        if key != 'data': 
            self.node[key] = val 
        else: 
            # val should be a dict 
            assert type(val) == dict
            self.node['data'].update(val) 
            
            
class KnowledgeGraph: 
    
    def __init__(self): 
        self.graph = [] 
    
    def addNode(self, node, multiple=False):  
        if multiple: 
            # node should be a list
            self.graph.extend(node) 
        else:
            self.graph.append(node)

def try_node_id(d, p): 
    try: 
        r = d.get(p).node['uid']
    except: 
        r = None
    return r

class Query: 
    
    def __init__(self, graph):  
        try:
            self.graph = [x.node['data'] for x in graph.graph] 
        except: 
            self.graph = graph
    
    
    def query(self, triples): 
        """
        subject is (str) -- from instance of 
        predicate is str -- should be key
        obj is node 
        """ 
        
        """
        triples is list of tuples like ('human', 'cousin to', bob_node)
        """ 
        output = self.graph
        for s,p,o in triples: 
            
            output = [x for x in output if (x.get('instance of') == s) and (try_node_id(x,p) == o.node.get('uid'))]
            
        return output 
        
#         if type(obj) != dict: 
#             obj = obj.node
    
#         out = [x for x in self.graph if (x.get('instance of') == subject) and (x.get(predicate).node['uid'] == obj.get('uid'))]
#         return out 

### task: create a family tree knowledge graph and be able to query it 

--> person nodes can (but don't have to) include name, famillial relations (cousin, son, etc), and job titles 


In [23]:
# 1. create nodes 

bob = Node({
    'name': 'bob', 
    'instance of': 'human',
    'cousin to': 'larry', 
    'son of': 'donald', 
    'occupation': 'teacher'
})

larry = Node({
    'name': 'larry',  
    'instance of': 'human',
    'cousin to': 'bob', 
    'son of': 'howard', 
    'occupation': 'plumber'
})

donald = Node({
    'name': 'donald',  
    'instance of': 'human',
    'father to': 'bob', 
    'brother to': 'howard', 
    'occupation': 'retired'
})

howard = Node({
    'name': 'howard',  
    'instance of': 'human',
    'father to': 'larry', 
    'brother to': 'donald', 
    'occupation': 'retired'
})

teacher = Node({
    'name': 'teacher', 
    'instance of': 'occupation', 
    'pay': '$50,000', 
    'pay type': 'salary'
}) 

plumber = Node({
    'name': 'plumber', 
    'instance of': 'occupation', 
    'pay': '$40', 
    'pay type': 'hourly'
})

retired = Node({
    'name': 'retired', 
    'instance of': 'occupation', 
    'pay': '$0', 
    'pay type': None
})

# 2. alter nodes to include references to other nodes 

bob.alter('data', {'cousin to': larry})
bob.alter('data', {'occupation': teacher})
bob.alter('data', {'son of': donald})

larry.alter('data', {'cousin to': bob})
larry.alter('data', {'occupation': plumber})
larry.alter('data', {'son of': howard})

donald.alter('data', {'father to': bob})
donald.alter('data', {'occupation': retired})
donald.alter('data', {'brother to': howard})

howard.alter('data', {'father to': larry})
howard.alter('data', {'occupation': retired})
howard.alter('data', {'brother to': donald})


In [24]:
kg = KnowledgeGraph()
kg.addNode([bob, larry, donald, howard, teacher, plumber, retired], multiple=True)

In [28]:
engine = Query(kg)
engine.query([('human', 'occupation', retired), ('human', 'father to', bob)])

[{'name': 'donald',
  'instance of': 'human',
  'father to': <__main__.Node at 0x7fe9e1905bb0>,
  'brother to': <__main__.Node at 0x7fe9e1905f10>,
  'occupation': <__main__.Node at 0x7fe9e1905700>}]